In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from KCrossVal import kFoldCross, MeanAbsError, BinaryAccuracy
import time

In [109]:
dataset_df = pd.read_csv('./data/Financial_Data.csv')
dataset_df = dataset_df.sample(frac=1)
excluded_cols = ['Stock', 'Sector', 'Class', 'Year', 'PRICE VAR [%]']
num_cols = [c for c in dataset_df.columns if c not in excluded_cols]
X_dev = dataset_df[num_cols].to_numpy()
Y_float = dataset_df['PRICE VAR [%]'].to_numpy()
Y_bin = dataset_df['Class'].to_numpy()

print(len(X_dev))

print(np.sum((Y_float>100).astype(int)))

Y_float[Y_float>100] = 100
#not_outliers = np.abs(Y_float - Y_float.mean()) < Y_float.std()*0.01

#X_dev = X_dev[not_outliers]
#Y_float = Y_float[not_outliers]
#Y_bin = Y_bin[not_outliers]



19500
735


In [102]:
np.atleast_2d(Y_float).transpose()

array([[ 13.97256543],
       [100.        ],
       [-14.75211849],
       ...,
       [-27.55555577],
       [ 50.04483774],
       [ 58.25926291]])

In [110]:
C_values = [1]
results = []
for C in C_values:
    reg = linear_model.LogisticRegression(C=C,max_iter=1000)
    res = kFoldCross(reg.fit, reg.predict, X_dev, Y_bin, 5, MeanAbsError)
    res = C, res[0], res[1], res[3]
    results.append(res)

df = pd.DataFrame(results, columns=['C', 'ValErr_mean', 'ValErr_std', 'seconds'])
df.sort_values(by='ValErr_mean')

,C,ValErr_mean,ValErr_std,seconds
0,1,0.426,0.003643,40.759921


In [112]:
alpha = [1,10,20,30]
results = []
for a in alpha:
    reg = linear_model.Ridge(alpha=a)
    res = kFoldCross(reg.fit, reg.predict, X_dev, Y_float, 5, MeanAbsError)
    res = a, res[0], res[1], res[3]
    results.append(res)

df = pd.DataFrame(results, columns=['alpha', 'ValErr_mean', 'ValErr_std', 'seconds'])
df.sort_values(by='ValErr_mean')

,alpha,ValErr_mean,ValErr_std,seconds
3,30,32.205939,0.644349,2.011663
2,20,32.261710,0.653381,1.932627
1,10,32.399098,0.702327,1.690888
0,1,33.809089,2.117411,1.606152


In [107]:
corr_values = []
for c in num_cols:
    array = dataset_df[c]
    corr = np.corrcoef(array, Y_float)[0,1]
    corr = np.abs(corr)
    corr_values.append((corr, c))

corr_values.sort(key= lambda tup: tup[0])
corr_values

[(0.00017916845476845518, 'Free Cash Flow'),
 (0.0002397917396880654, 'Dividend Yield'),
 (0.0003884306262740216, 'Operating Cash Flow'),
 (0.00045712718351258446, 'enterpriseValueMultiple'),
 (0.00045760014145230035, 'longtermDebtToCapitalization'),
 (0.00047185938642697753, 'Total non-current assets'),
 (0.0005059434776818749, 'Operating Cash Flow growth'),
 (0.0005534954298999023, 'Gross Margin'),
 (0.0007256594522473231, 'Interest Expense'),
 (0.0007958823409593013, 'EPS Diluted Growth'),
 (0.001325136300765979, 'priceEarningsRatio'),
 (0.0013640023692383855, 'priceFairValue'),
 (0.0014564882360195892, 'Acquisitions and disposals'),
 (0.0015074749660009242, 'Weighted Average Shares Growth'),
 (0.001677515310548997, 'freeCashFlowPerShare'),
 (0.0018002224894944328, 'Preferred Dividends'),
 (0.0018820174637722308, 'Other Assets'),
 (0.0022155635887927323, 'EBITDA Margin'),
 (0.002295041785543817, 'Depreciation & Amortization'),
 (0.0023252004160653227, 'R&D Expenses'),
 (0.0025100838

In [33]:
print(BinaryAccuracy(X_dev, Y_dev2-0.5))
print(BinaryAccuracy(X_dev, Y_dev1))
print(BinaryAccuracy(Y_dev2, Y_dev1))

0.4577019171597633
0.4584464378698225
0.0008205128205128205


In [106]:
np.corrcoef(dataset_df['Receivables'].to_numpy(), Y_bin)[0,1]

-0.007444499026286381

In [52]:
(Y_dev1.std(), Y_dev1.mean())

(18273.53745234518, 228.3855775263013)

Y_dev2.mean()